In [11]:
import pandas as pd
import sys
from pathlib import Path
import numpy as np
from itertools import product
from tqdm import tqdm
# sys.path.append('/data/gpfs/projects/punim1395/workspace/DeepCTR-Torch')
# from ..deepctr_torch.layers.utils import slice_arrays
!pwd

/data/gpfs/projects/punim1395/workspace/dataset/Amazon_review/scripts


In [12]:
csvpath = Path('../small_subsets/Movies_and_TV/reviews_Movies_and_TV_5.csv')
csvpath_noname = csvpath.parent

In [13]:
csvpath_noname

PosixPath('../small_subsets/Movies_and_TV')

In [14]:
meta_path = Path('../small_subsets/Movies_and_TV/\
meta_Movies_and_TV_valid.csv')
meta_Movies_and_TV = pd.read_csv(meta_path).iloc[:, 1:]
ap_Movies_and_TV = meta_Movies_and_TV[['asin', 'path']]

/home/liangchenl/.conda/envs/jupyterlab-debugger/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
csv_Movies_and_TV_5 = pd.read_csv(csvpath)

In [16]:
csv_Movies_and_TV_5.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,ADZPIG9QOCDG5,0005019281,"Alice L. Larson ""alice-loves-books""","[0, 0]",This is a charming version of the classic Dick...,4.0,good version of a classic,1203984000,"02 26, 2008"
1,1,A35947ZP82G7JH,0005019281,Amarah Strack,"[0, 0]",It was good but not as emotionally moving as t...,3.0,Good but not as moving,1388361600,"12 30, 2013"
2,2,A3UORV8A9D5L2E,0005019281,Amazon Customer,"[0, 0]","Don't get me wrong, Winkler is a wonderful cha...",3.0,Winkler's Performance was ok at best!,1388361600,"12 30, 2013"
3,3,A1VKW06X1O2X7V,0005019281,"Amazon Customer ""Softmill""","[0, 0]",Henry Winkler is very good in this twist on th...,5.0,It's an enjoyable twist on the classic story,1202860800,"02 13, 2008"
4,4,A3R27T4HADWFFJ,0005019281,BABE,"[0, 0]",This is one of the best Scrooge movies out. H...,4.0,Best Scrooge yet,1387670400,"12 22, 2013"


In [18]:
print(len(csv_Movies_and_TV_5))
print(len(ap_Movies_and_TV))

1697533
198933


In [91]:
csv_Movies_and_TV_5.tail()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Binary_rating
1697528,1697528,AV657BUYHHXZ2,B00LT1JHLW,"Mike Rules ""Mike""","[1, 14]",wow $269.99 for the entire series on Blu Ray??...,1.0,Way to Expensive!! WB = GREED,1406073600,"07 23, 2014",False
1697529,1697529,A17W587EH23J0Q,B00LT1JHLW,"Ron2900 ""Ron""","[32, 48]","Finally, the holy grail of tv-on-dvd boxsets i...",5.0,"HOLY BAT-BOXSET, BATMAN... I never thought thi...",1405641600,"07 18, 2014",True
1697530,1697530,A3DE438TF1A958,B00LT1JHLW,thomas henry,"[3, 10]",Could this be a true or I'm i dreaming batman ...,5.0,prayers have been answered because batman 60s ...,1405728000,"07 19, 2014",True
1697531,1697531,A2RWCXDMANY0LW,B00LT1JHLW,wheev,"[0, 4]",I've been a fan of the series since I was a yo...,5.0,can't Wait!,1405987200,"07 22, 2014",True
1697532,1697532,A3ROPC55BE2OM9,B00LT1JHLW,WingLT,"[11, 23]",People seriously need to wake up and realize t...,5.0,The Price is Insane? People Really Need to Wak...,1405728000,"07 19, 2014",True


In [85]:
csv_Movies_and_TV_5['Binary_rating'] = csv_Movies_and_TV_5['overall'].astype(int)>2

In [106]:
num_pos = len(csv_Movies_and_TV_5[csv_Movies_and_TV_5['Binary_rating'] == True]);
num_pos

1490904

In [97]:
num_neg = len(csv_Movies_and_TV_5[csv_Movies_and_TV_5['Binary_rating'] == False]); num_neg

206629

In [103]:
num_diff = num_pos - num_neg; num_diff

1284275

In [109]:
rID_all = np.unique(csv_Movies_and_TV_5['reviewerID'])
asin_all = np.unique(csv_Movies_and_TV_5['asin'])

In [112]:
num_diff = num_pos - num_neg
num_csv_Movies_and_TV_5_balance = len(csv_Movies_and_TV_5)+num_diff
ra_all = np.column_stack((np.array(csv_Movies_and_TV_5['reviewerID']),
                          np.array(csv_Movies_and_TV_5['asin'])))
ra_all = ra_all.astype(str)
ra_all_balance = ra_all
while True:
    rID_extra_index = np.random.choice(len(rID_all), num_diff)
    rID_extra = rID_all[rID_extra_index]
    rID_extra = rID_extra.astype(str)
    asin_extra_index = np.random.choice(len(asin_all), num_diff)
    asin_extra = asin_all[asin_extra_index]
    asin_extra = asin_extra.astype(str)
    ra_extra_index = np.column_stack((rID_extra_index, asin_extra_index))
    ra_extra = np.column_stack((rID_extra, asin_extra))
    ra_all_balance = np.concatenate((ra_all_balance, ra_extra))
    ra_all_balance, ra_index = np.unique(ra_all_balance, axis=0, return_index=True)
    ra_all_balance = ra_all_balance[np.argsort(ra_index)]
    if len(ra_all_balance) < num_csv_Movies_and_TV_5_balance:
        num_diff = num_csv_Movies_and_TV_5_balance - len(ra_all_balance)
    else:
        break

In [115]:
sample_extra = np.array([0.]*(num_pos - num_neg))
overall_col = np.array(csv_Movies_and_TV_5['overall'])
overall_col = np.concatenate([overall_col, sample_extra])
overall_col = overall_col.astype('int')

In [130]:
ra_all_balance

array([['ADZPIG9QOCDG5', '0005019281'],
       ['A35947ZP82G7JH', '0005019281'],
       ['A3UORV8A9D5L2E', '0005019281'],
       ...,
       ['A38EGLEZ5CFVAT', 'B0013JKXA4'],
       ['A39N0BN4B2Q7F', 'B00005TNFY'],
       ['A10FLVDK4WFFDQ', 'B003VMFWXO']], dtype='<U21')

In [122]:
rao_all_balance = ra_all_balance

In [139]:
rao_all_balance = np.column_stack((rao_all_balance, overall_col.astype(np.int8)))

In [142]:
rao_all_balance[6]

array(['A5NYUBEKXFLX5', '0005019281', '5'], dtype='<U21')

In [145]:
df = pd.DataFrame(rao_all_balance, columns = ['reviewerID', 'asin', 'overall'])

In [151]:
df['Binary_rating'] = df['overall'].astype(int)>2

In [161]:
rao_all_balance_path = csvpath_noname/'rao_all_5_balance.csv'

In [164]:
df.to_csv(rao_all_balance_path)

In [245]:
df.head()

,reviewerID,asin,overall,Binary_rating
0,ADZPIG9QOCDG5,0005019281,4,True
1,A35947ZP82G7JH,0005019281,3,True
2,A3UORV8A9D5L2E,0005019281,3,True
3,A1VKW06X1O2X7V,0005019281,5,True
4,A3R27T4HADWFFJ,0005019281,4,True


In [200]:
df_random_raob = df.sample(frac=1) 
# raob means rID asin overall Binary_rating

In [242]:
df_random_raob.head()

,reviewerID,asin,overall,Binary_rating
1044335,AQ9KPZLITYP3M,B000M9BPGM,1,False
2957855,A3I1VYV6NN83WR,B00006IUFT,0,False
2282437,A81TG8ZBUFAK4,630196019X,0,False
2165023,A2DDKFU4NDJ15K,B00005TNEY,0,False
1192002,A6J7UPYUI96XO,B001FQLMTQ,3,True


In [206]:
meta_path = Path('../small_subsets/Movies_and_TV/\
meta_Movies_and_TV_valid.csv')
meta_Movies_and_TV = pd.read_csv(meta_path).iloc[:, 1:]
ap_Movies_and_TV = meta_Movies_and_TV[['asin', 'path']]

/home/liangchenl/.conda/envs/jupyterlab-debugger/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [209]:
len(df_random_raob)

2981808

In [212]:
df_random_raobp = df_random_raob.\
merge(ap_Movies_and_TV, how='left', on='asin')

In [272]:
df_random_raobp_dropna = df_random_raobp[df_random_raobp['path'].notna()]

In [281]:
len(df_random_raobp_dropna)

2925234

In [284]:
df_random_raobp_dropna.head()

,reviewerID,asin,overall,Binary_rating,path
0,AQ9KPZLITYP3M,B000M9BPGM,1,False,../small_subsets/Movies_and_TV/img/B000M9BPGM.jpg
1,A3I1VYV6NN83WR,B00006IUFT,0,False,../small_subsets/Movies_and_TV/img/B00006IUFT.jpg
2,A81TG8ZBUFAK4,630196019X,0,False,../small_subsets/Movies_and_TV/img/630196019X.jpg
3,A2DDKFU4NDJ15K,B00005TNEY,0,False,../small_subsets/Movies_and_TV/img/B00005TNEY.jpg
4,A6J7UPYUI96XO,B001FQLMTQ,3,True,../small_subsets/Movies_and_TV/img/B001FQLMTQ.jpg


In [287]:
val_ratio = 0.1
test_ratio = 0.1

In [290]:
if hasattr(df_random_raobp_dropna, 'shape'):
    val_split_at = int(df_random_raobp_dropna.shape[0] * (1. - val_ratio-test_ratio))
    test_split_at = int(df_random_raobp_dropna.shape[0] * (1. - test_ratio))
else:
    val_split_at = int(len(df_random_raobp_dropna) * (1. - val_ratio-test_ratio))
    test_split_at = int(len(df_random_raobp_dropna) * (1. - test_ratio))

In [293]:
df_random_raobp_train = df_random_raobp_dropna[:val_split_at]
df_random_raobp_val = df_random_raobp_dropna[val_split_at:test_split_at]
df_random_raobp_test = df_random_raobp_dropna[test_split_at:]

In [296]:
df_random_raobp_train.head()

,reviewerID,asin,overall,Binary_rating,path
0,AQ9KPZLITYP3M,B000M9BPGM,1,False,../small_subsets/Movies_and_TV/img/B000M9BPGM.jpg
1,A3I1VYV6NN83WR,B00006IUFT,0,False,../small_subsets/Movies_and_TV/img/B00006IUFT.jpg
2,A81TG8ZBUFAK4,630196019X,0,False,../small_subsets/Movies_and_TV/img/630196019X.jpg
3,A2DDKFU4NDJ15K,B00005TNEY,0,False,../small_subsets/Movies_and_TV/img/B00005TNEY.jpg
4,A6J7UPYUI96XO,B001FQLMTQ,3,True,../small_subsets/Movies_and_TV/img/B001FQLMTQ.jpg


In [299]:
df_random_raobp_train.to_csv(csvpath_noname/'raobp_Movies_and_TV_5_random_08_train_binary_balance.csv')
# save the 0.8 of rID asin and overall train for training and fair compare

In [302]:
df_random_raobp_val.to_csv(csvpath_noname/'raobp_Movies_and_TV_5_random_01_val_binary_balance.csv') 
# save the 0.1 of rID asin and overall val for validation and fair compare

In [303]:
df_random_raobp_test.to_csv(csvpath_noname/'raobp_Movies_and_TV_5_random_01_test_binary_balance.csv') 
# save the 0.1 of rID asin and overall test for test and fair compare